In [48]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np

In [49]:
census = pd.read_csv('../data/demographic_census.csv')
voting_turnout = pd.read_csv('../data/2018-turnout-population-congress-districts.csv', nrows = 436)
voting_age = pd.read_csv('../data/2018-age-population-congress-districts.csv', nrows = 436)
voting_ed = pd.read_csv('../data/2018-education-population-congress-districts.csv', nrows = 436)
voting_sex_poverty = pd.read_csv('../data/2018-sex-poverty-population-congress-districts.csv', nrows = 436)
voting_race = pd.read_csv('../data/2018-race-population-congress-districts.csv', nrows = 436)

census.head()

census.shape

## Remove excess columns. Create new column that combines State Abbreviation and Congressional District, named District (for easier merges later). Convert object columns to either int64 or float datatypes. 

## Voter Turnout data clean up. 

In [ ]:
voting_turnout.head()

In [50]:
voting_turnout = voting_turnout.drop(columns = ['Line #', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                              'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'])

In [51]:
voting_turnout['District'] = voting_turnout['State abbreviation'] + "-" + voting_turnout['Congressional district']

In [52]:
voting_turnout.columns

Index(['State abbreviation', 'State name', 'Congressional district',
       'Votes cast for congressional representative for the November 6, 2018 election1',
       ' Citizen voting-age population Estimate ',
       ' Citizen voting-age population Margin of error (MOE) ',
       ' Voting Rate Estimate ', ' Voting Rate Margin of error (MOE) ',
       'District'],
      dtype='object')

In [53]:
voting_turnout = voting_turnout[['District', 'State abbreviation', 'State name', 'Congressional district',
       'Votes cast for congressional representative for the November 6, 2018 election1',
       ' Citizen voting-age population Estimate ',
       ' Citizen voting-age population Margin of error (MOE) ',
       ' Voting Rate Estimate ', ' Voting Rate Margin of error (MOE) ']]

In [54]:
voting_turnout.columns = ['District', 'State abbreviation', 'State name', 'Congressional district',
       'Votes cast for congressional reps for the Nov 2018 election',
       'Citizen voting-age population Estimate',
       'Citizen voting-age population MOE',
       'Voting Rate Estimate', 'Voting Rate MOE']

In [55]:
voting_turnout.head()

,District,State abbreviation,State name,Congressional district,Votes cast for congressional reps for the Nov 2018 election,Citizen voting-age population Estimate,Citizen voting-age population MOE,Voting Rate Estimate,Voting Rate MOE
0,AL-1,AL,Alabama,1,"242,617","544,464","3,424",44.6,0.3
1,AL-2,AL,Alabama,2,"226,230","516,295","5,674",43.8,0.5
2,AL-3,AL,Alabama,3,"231,915","543,854","4,099",42.6,0.3
3,AL-4,AL,Alabama,4,"230,969","515,701","4,678",44.8,0.4
4,AL-5,AL,Alabama,5,"260,673","551,968","2,121",47.2,0.2


In [ ]:
voting_turnout.describe

In [ ]:
voting_turnout.dtypes

In [56]:
voting_turnout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 9 columns):
 #   Column                                                       Non-Null Count  Dtype 
---  ------                                                       --------------  ----- 
 0   District                                                     436 non-null    object
 1   State abbreviation                                           436 non-null    object
 2   State name                                                   436 non-null    object
 3   Congressional district                                       436 non-null    object
 4   Votes cast for congressional reps for the Nov 2018 election  436 non-null    object
 5   Citizen voting-age population Estimate                       436 non-null    object
 6   Citizen voting-age population MOE                            436 non-null    object
 7   Voting Rate Estimate                                         436 non-null    object
 8   V

In [57]:
voting_turnout.tail()

,District,State abbreviation,State name,Congressional district,Votes cast for congressional reps for the Nov 2018 election,Citizen voting-age population Estimate,Citizen voting-age population MOE,Voting Rate Estimate,Voting Rate MOE
431,WI-5,WI,Wisconsin,5,"364,288","558,805","5,239",65.2,0.6
432,WI-6,WI,Wisconsin,6,"325,065","556,485","2,532",58.4,0.3
433,WI-7,WI,Wisconsin,7,"322,840","552,651","2,556",58.4,0.3
434,WI-8,WI,Wisconsin,8,"328,774","556,205","2,262",59.1,0.2
435,WY-At-large,WY,Wyoming,At-large,"205,275","433,644","2,221",47.3,0.2


In [28]:
voting_turnout.shape

(436, 19)

In [61]:
type(voting_turnout)

pandas.core.frame.DataFrame

In [59]:
voting_turnout['Votes cast for congressional reps for the Nov 2018 election'].str.replace(',', '').astype(int)

ValueError: invalid literal for int() with base 10: ' -   '

In [60]:
voting_turnout['Votes cast for congressional reps for the Nov 2018 election'].str.replace(',', '').astype(int)
#print(voting_turnout.dtypes)

ValueError: invalid literal for int() with base 10: ' -   '

In [ ]:
convert_dict = {'Votes cast for congressional reps for the Nov 2018 election': int,
       'Citizen voting-age population Estimate': int,
       'Citizen voting-age population MOE': int,
       'Voting Rate Estimate': float, 'Voting Rate MOE': float}

In [ ]:
voting_turnout = voting_turnout.astype(convert_dict)
#print(voting_turnout.dtypes)

## method 1 
df["a"] = df["a"].astype(str).astype(int)
print(df.dtypes)

## method 2 
# importing pandas as pd 
import pandas as pd 
  
# sample dataframe 
df = pd.DataFrame({ 
    'A': [1, 2, 3, 4, 5], 
    'B': ['a', 'b', 'c', 'd', 'e'], 
    'C': [1.1, '1.0', '1.3', 2, 5] }) 
  
# using dictionary to convert specific columns 
convert_dict = {'A': int, 
                'C': float
               } 
  
df = df.astype(convert_dict) 
print(df.dtypes) 

# importing pandas as pd 
import pandas as pd 

##  method 3
# sample dataframe 
df = pd.DataFrame({ 
    'A': [1, 2, 3, '4', '5'], 
    'B': ['a', 'b', 'c', 'd', 'e'], 
    'C': [1.1, '2.1', 3.0, '4.1', '5.1'] }) 
  
# using apply method 
df[['A', 'C']] = df[['A', 'C']].apply(pd.to_numeric) 
print(df.dtypes) 

## Voters by Age data clean up. 

In [ ]:
voting_age.head()

In [ ]:
voting_age.info()

In [ ]:
voting_age.columns

In [36]:
voting_age = voting_age[['State abbreviation',
                                 'State name',
                                 'Congressional district',
                                 ' Citizen voting-age population Estimate ',
                                 ' Citizen voting-age population Margin of error (MOE) ',
                                 ' 18-29 Age Estimate ',
                                 ' 18-29 Age  Margin of error (MOE) ',
                                 ' Age 18-29 Percent of total ',
                                 ' 18-29 Age Percent of total margin of error (MOE) ',
                                 ' 30-44 Age Estimate ',
                                 ' 30-44 Age Margin of error (MOE) ',
                                 ' 30-44 Age Percent of total ',
                                 ' 30-44 Age Percent of total margin of error (MOE) ',
                                 ' 45-64 Age Estimate ',
                                 ' 45-64 Age Margin of error (MOE) ',
                                 ' 45-64 Age Percent of total ',
                                 ' 45-64 Age Percent of total margin of error (MOE) ',
                                 ' 65+ Age Estimate ',
                                 ' 65+ Age Margin of error (MOE) ',
                                 ' 65+ Age Percent of total ',
                                 ' 65+ Age Percent of total margin of error (MOE) ']]

In [39]:
voting_age['District'] = voting_age['State abbreviation'] + "-" + voting_age['Congressional district']

In [40]:
voting_age.head()

,State abbreviation,State name,Congressional district,Citizen voting-age population Estimate,Citizen voting-age population Margin of error (MOE),18-29 Age Estimate,18-29 Age Margin of error (MOE),Age 18-29 Percent of total,18-29 Age Percent of total margin of error (MOE),30-44 Age Estimate,...,30-44 Age Percent of total margin of error (MOE),45-64 Age Estimate,45-64 Age Margin of error (MOE),45-64 Age Percent of total,45-64 Age Percent of total margin of error (MOE),65+ Age Estimate,65+ Age Margin of error (MOE),65+ Age Percent of total,65+ Age Percent of total margin of error (MOE),District
0,AL,Alabama,1,"544,464","3,424","104,060","2,412",19.1,0.4,"125,900",...,0.5,"188,031","3,186",34.5,0.5,"126,473","1,797",23.2,0.3,AL-1
1,AL,Alabama,2,"516,295","5,674","109,222","2,918",21.2,0.5,"118,999",...,0.5,"173,351","3,069",33.6,0.5,"114,723","2,059",22.2,0.4,AL-2
2,AL,Alabama,3,"543,854","4,099","121,328","2,379",22.3,0.4,"121,212",...,0.6,"182,379","3,899",33.5,0.7,"118,935","1,851",21.9,0.3,AL-3
3,AL,Alabama,4,"515,701","4,678","95,990","2,777",18.6,0.5,"114,921",...,0.5,"177,247","3,022",34.4,0.5,"127,543","2,309",24.7,0.4,AL-4
4,AL,Alabama,5,"551,968","2,121","109,986","1,624",19.9,0.3,"129,788",...,0.3,"193,348","1,995",35.0,0.4,"118,846","1,412",21.5,0.3,AL-5


In [35]:
voting_age.columns

Index(['State abbreviation', 'State name', 'Congressional district',
       ' Citizen voting-age population Estimate ',
       ' Citizen voting-age population Margin of error (MOE) ',
       ' 18-29 Age Estimate ', ' 18-29 Age  Margin of error (MOE) ',
       ' Age 18-29 Percent of total ',
       ' 18-29 Age Percent of total margin of error (MOE) ',
       ' 30-44 Age Estimate ', ' 30-44 Age Margin of error (MOE) ',
       ' 30-44 Age Percent of total ',
       ' 30-44 Age Percent of total margin of error (MOE) ',
       ' 45-64 Age Estimate ', ' 45-64 Age Margin of error (MOE) ',
       ' 45-64 Age Percent of total ',
       ' 45-64 Age Percent of total margin of error (MOE) ',
       ' 65+ Age Estimate ', ' 65+ Age Margin of error (MOE) ',
       ' 65+ Age Percent of total ',
       ' 65+ Age Percent of total margin of error (MOE) ', 'District'],
      dtype='object')

In [41]:
voting_age = voting_age[['District', 'State abbreviation', 'State name', 'Congressional district',
       ' Citizen voting-age population Estimate ',
       ' Citizen voting-age population Margin of error (MOE) ',
       ' 18-29 Age Estimate ', ' 18-29 Age  Margin of error (MOE) ',
       ' Age 18-29 Percent of total ',
       ' 18-29 Age Percent of total margin of error (MOE) ',
       ' 30-44 Age Estimate ', ' 30-44 Age Margin of error (MOE) ',
       ' 30-44 Age Percent of total ',
       ' 30-44 Age Percent of total margin of error (MOE) ',
       ' 45-64 Age Estimate ', ' 45-64 Age Margin of error (MOE) ',
       ' 45-64 Age Percent of total ',
       ' 45-64 Age Percent of total margin of error (MOE) ',
       ' 65+ Age Estimate ', ' 65+ Age Margin of error (MOE) ',
       ' 65+ Age Percent of total ',
       ' 65+ Age Percent of total margin of error (MOE) ']]

In [42]:
voting_age.columns = ['District', 'State abbreviation', 'State name', 'Congressional district',
       'Citizen voting-age population Estimate',
       'Citizen voting-age population MOE',
       '18-29 Age Estimate', '18-29 Age MOE',
       '18-29 Age Percent of total',
       '18-29 Age Percent of total MOE',
       '30-44 Age Estimate', '30-44 Age MOE',
       '30-44 Age Percent of total',
       '30-44 Age Percent of total MOE',
       '45-64 Age Estimate', '45-64 Age MOE',
       '45-64 Age Percent of total',
       '45-64 Age Percent of total MOE',
       '65+ Age Estimate', '65+ Age MOE',
       '65+ Age Percent of total',
       '65+ Age Percent of total MOE']

In [43]:
voting_age.head()

,District,State abbreviation,State name,Congressional district,Citizen voting-age population Estimate,Citizen voting-age population MOE,18-29 Age Estimate,18-29 Age MOE,18-29 Age Percent of total,18-29 Age Percent of total MOE,...,30-44 Age Percent of total,30-44 Age Percent of total MOE,45-64 Age Estimate,45-64 Age MOE,45-64 Age Percent of total,45-64 Age Percent of total MOE,65+ Age Estimate,65+ Age MOE,65+ Age Percent of total,65+ Age Percent of total MOE
0,AL-1,AL,Alabama,1,"544,464","3,424","104,060","2,412",19.1,0.4,...,23.1,0.5,"188,031","3,186",34.5,0.5,"126,473","1,797",23.2,0.3
1,AL-2,AL,Alabama,2,"516,295","5,674","109,222","2,918",21.2,0.5,...,23.0,0.5,"173,351","3,069",33.6,0.5,"114,723","2,059",22.2,0.4
2,AL-3,AL,Alabama,3,"543,854","4,099","121,328","2,379",22.3,0.4,...,22.3,0.6,"182,379","3,899",33.5,0.7,"118,935","1,851",21.9,0.3
3,AL-4,AL,Alabama,4,"515,701","4,678","95,990","2,777",18.6,0.5,...,22.3,0.5,"177,247","3,022",34.4,0.5,"127,543","2,309",24.7,0.4
4,AL-5,AL,Alabama,5,"551,968","2,121","109,986","1,624",19.9,0.3,...,23.5,0.3,"193,348","1,995",35.0,0.4,"118,846","1,412",21.5,0.3


In [44]:
voting_age.tail()

,District,State abbreviation,State name,Congressional district,Citizen voting-age population Estimate,Citizen voting-age population MOE,18-29 Age Estimate,18-29 Age MOE,18-29 Age Percent of total,18-29 Age Percent of total MOE,...,30-44 Age Percent of total,30-44 Age Percent of total MOE,45-64 Age Estimate,45-64 Age MOE,45-64 Age Percent of total,45-64 Age Percent of total MOE,65+ Age Estimate,65+ Age MOE,65+ Age Percent of total,65+ Age Percent of total MOE
431,WI-5,WI,Wisconsin,5,"558,805","5,239","98,106","2,914",17.6,0.5,...,23.6,0.5,"197,688","3,537",35.4,0.5,"131,160","2,994",23.5,0.5
432,WI-6,WI,Wisconsin,6,"556,485","2,532","102,408","1,665",18.4,0.3,...,22.0,0.4,"200,105","2,042",36.0,0.3,"131,640","1,068",23.7,0.2
433,WI-7,WI,Wisconsin,7,"552,651","2,556","86,048","1,610",15.6,0.3,...,21.0,0.3,"206,377","1,798",37.3,0.3,"144,269","1,494",26.1,0.2
434,WI-8,WI,Wisconsin,8,"556,205","2,262","101,057","1,787",18.2,0.3,...,23.2,0.3,"200,720","1,671",36.1,0.3,"125,516","1,064",22.6,0.2
435,WY-At-large,WY,Wyoming,At-large,"433,644","2,221","86,967","1,666",20.1,0.4,...,24.5,0.4,"144,337","2,205",33.3,0.5,"95,979","1,485",22.1,0.3


## Voter Education data clean up. 

In [45]:
voting_ed.head()

,Line #,State abbreviation,State name,Congressional district,Citizen voting-age population Estimate,Citizen voting-age population Margin of error (MOE),Less than 9th grade Estimate,Less than 9th grade Margin of error (MOE),Less than 9th grade Percent of total,Less than 9th grade Percent of total margin of error (MOE),...,Graduate degree Percent of total margin of error (MOE),High school or more Estimate,High school or more Margin of error (MOE),High school or more Percent of total,High school or more Percent of total margin of error (MOE),Bachelor's degree or more Estimate,Bachelor's degree or more Margin of error (MOE),Bachelor's degree or more Percent of total,Bachelor's degree or more Percent of total margin of error (MOE),Unnamed: 42
0,1,AL,Alabama,1,"544,464","3,424","13,409","1,845",2.5,0.3,...,0.7,"473,821","5,794",87.0,0.9,"123,479","5,614",22.7,1.1,NaN
1,2,AL,Alabama,2,"516,295","5,674","18,295","2,059",3.5,0.4,...,0.6,"444,808","6,761",86.2,0.9,"107,354","5,009",20.8,1.0,NaN
2,3,AL,Alabama,3,"543,854","4,099","19,578","2,504",3.6,0.5,...,0.7,"467,443","6,717",86.0,1.0,"104,816","6,147",19.3,1.1,NaN
3,4,AL,Alabama,4,"515,701","4,678","24,346","2,997",4.7,0.6,...,0.5,"433,505","6,247",84.1,1.0,"84,524","4,487",16.4,0.8,NaN
4,5,AL,Alabama,5,"551,968","2,121","14,908","2,282",2.7,0.4,...,0.8,"493,255","5,090",89.4,0.8,"173,277","6,244",31.4,1.1,NaN


In [63]:
voting_ed = voting_ed.drop(columns = ['Line #', 'Unnamed: 42'])

In [64]:
voting_ed['District'] = voting_ed['State abbreviation'] + "-" + voting_ed['Congressional district']

In [65]:
voting_ed.columns

Index(['State abbreviation', 'State name', 'Congressional district',
       ' Citizen voting-age population Estimate ',
       ' Citizen voting-age population Margin of error (MOE) ',
       ' Less than 9th grade Estimate ',
       ' Less than 9th grade Margin of error (MOE) ',
       ' Less than 9th grade Percent of total ',
       ' Less than 9th grade Percent of total margin of error (MOE) ',
       ' 9th to 12 Grade, no diploma Estimate ',
       ' 9th to 12 Grade, no diploma Margin of error (MOE) ',
       ' 9th to 12 Grade, no diploma Percent of total ',
       ' 9th to 12 Grade, no diploma Percent of total margin of error (MOE) ',
       ' High school graduate Estimate ',
       ' High school graduate Margin of error (MOE) ',
       ' High school graduate Percent of total ',
       ' High school graduate Percent of total margin of error (MOE) ',
       ' Some college, no degree Estimate ',
       ' Some college, no degree Margin of error (MOE) ',
       ' Some college, no degree

In [71]:
voting_ed.columns = ['State abbreviation', 'State name', 'Congressional district',
       'Citizen voting-age population Estimate',
       'Citizen voting-age population MOE',
       'Less than 9th grade Estimate',
       'Less than 9th grade MOE',
       'Less than 9th grade Percent of total',
       'Less than 9th grade Percent of total MOE',
       '9th to 12 Grade, no diploma Estimate',
       '9th to 12 Grade, no diploma MOE',
       '9th to 12 Grade, no diploma Percent of total',
       '9th to 12 Grade, no diploma Percent of total MOE',
       'High school graduate Estimate',
       'High school graduate MOE',
       'High school graduate Percent of total',
       'High school graduate Percent of total MOE',
       'Some college, no degree Estimate',
       'Some college, no degree MOE',
       'Some college, no degree Percent of total',
       'Some college, no degree Percent of total MOE',
       'Associates degree Estimate',
       'Associates degree MOE',
       'Associates degree Percent of total',
       'Associates degree Percent of total MOE',
       'Bachelors degree Estimate',
       'Bachelors degree MOE',
       'Bachelors degree Percent of total',
       'Bachelors degree Percent of total MOE',
       'Graduate degree Estimate', 'Graduate degree MOE',
       'Graduate degree Percent of total',
       'Graduate degree Percent of total MOE',
       'High school or more Estimate',
       'High school or more MOE',
       'High school or more Percent of total',
       'High school or more Percent of total MOE',
       'Bachelors degree or more Estimate',
       'Bachelors degree or more MOE',
       'Bachelors degree or more Percent of total',
       'Bachelors degree or more Percent of total MOE', 'District']


In [72]:
voting_ed.head()

,State abbreviation,State name,Congressional district,Citizen voting-age population Estimate,Citizen voting-age population MOE,Less than 9th grade Estimate,Less than 9th grade MOE,Less than 9th grade Percent of total,Less than 9th grade Percent of total MOE,"9th to 12 Grade, no diploma Estimate",...,Graduate degree Percent of total MOE,High school or more Estimate,High school or more MOE,High school or more Percent of total,High school or more Percent of total MOE,Bachelors degree or more Estimate,Bachelors degree or more MOE,Bachelors degree or more Percent of total,Bachelors degree or more Percent of total MOE,District
0,AL,Alabama,1,"544,464","3,424","13,409","1,845",2.5,0.3,"57,234",...,0.7,"473,821","5,794",87.0,0.9,"123,479","5,614",22.7,1.1,AL-1
1,AL,Alabama,2,"516,295","5,674","18,295","2,059",3.5,0.4,"53,192",...,0.6,"444,808","6,761",86.2,0.9,"107,354","5,009",20.8,1.0,AL-2
2,AL,Alabama,3,"543,854","4,099","19,578","2,504",3.6,0.5,"56,833",...,0.7,"467,443","6,717",86.0,1.0,"104,816","6,147",19.3,1.1,AL-3
3,AL,Alabama,4,"515,701","4,678","24,346","2,997",4.7,0.6,"57,850",...,0.5,"433,505","6,247",84.1,1.0,"84,524","4,487",16.4,0.8,AL-4
4,AL,Alabama,5,"551,968","2,121","14,908","2,282",2.7,0.4,"43,805",...,0.8,"493,255","5,090",89.4,0.8,"173,277","6,244",31.4,1.1,AL-5


## Voters by Sex and Poverty data clean up. 

In [ ]:
voting_sex_poverty.head()

In [ ]:
voting_sex_poverty.info()

In [ ]:
voting_sex_poverty = voting_sex_poverty.drop(columns = ['Line #', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 
                                              'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30'])

In [ ]:
voting_sex_poverty['District'] = voting_sex_poverty['State abbreviation'] + "-" + voting_sex_poverty['Congressional district']

In [ ]:
voting_sex_poverty.head()

## Voters by race data clean up. 

In [ ]:
voting_race.head()

In [ ]:
voting_race['District'] = voting_race['State abbreviation'] + "-" + voting_race['Congressional district']

In [ ]:
voting_race.head()

In [ ]:
alabama112 = gpd.read_file('../data/congressional-district-boundaries/Alabama_108_to_112.geojson')

In [ ]:
alabama112

In [ ]:
alabama112.plot();

In [ ]:
alabama112.columns

## read in geojson files for all congressional districts and append into one geodataframe. 

In [ ]:
districts_map = gpd.GeoDataFrame()

for i in range(1, 52):
    df = gpd.read_file("../data/congressional-districts/" + str(i) + ".geojson")
    districts_map = districts_map.append(df, ignore_index=True)

In [ ]:
df

In [ ]:
districts_map.tail(25)

In [ ]:
#districts_map.plot();
#figsize 20, 20
fig, ax = plt.subplots(figsize = (50, 10))
districts_map.plot(ax = ax)

ax.axis('off');


import os, json
#import pandas as pd

# this finds our json files
path_to_json = 'json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
#jsons_data = pd.DataFrame(columns=['country', 'city', 'long/lat'])

jsons_data = pd.DataFrame(columns=['startcong', 'district', 'statename', 'member', 'endcong', 'id', 'geometry'])

# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        country = json_text['features'][0]['properties']['country']
        city = json_text['features'][0]['properties']['name']
        lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [country, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)